# Results

In this notebook we explore the results of the experiments, which are stored in wandb

In [7]:
# Get wandb api object
import wandb

api = wandb.Api()

# Get all runs
project = api.project("pysentimiento")


# Get all runs for sentiment and lang es
runs = [r for r in api.runs("pysentimiento", {"config.lang": "it"}) if not r.sweep]

In [23]:
data = []

renames = {
    "Musixmatch/umberto-commoncrawl-cased-v1": "UmBERTo",
    "dbmdz/bert-base-italian-xxl-uncased": "BERT-it",
    "dbmdz/electra-base-italian-xxl-cased-discriminator": "Electra-it",
    "pysentimiento/robertuito-base-uncased": "RoBERTuito",
    "m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0": "AlBERTo",
}

for run in runs:
    # Get model name
    model_name = run.config["model"]
    # Get task 
    task = run.config["task"]
    # Get Macro f1 performance
    try:
        macro_f1 = run.summary["test_macro_f1"]
    except KeyError:
        print(f"Run {run.name} has no macro f1")
        print(run.summary)
        continue

    data.append({
        "model": model_name,
        "task": task,
        "macro_f1": macro_f1,
    })

Run true-elevator-360 has no macro f1
{'_wandb': {'runtime': 55}}
Run solar-armadillo-359 has no macro f1
{'_wandb': {'runtime': 61}}
Run deep-puddle-358 has no macro f1
{'_wandb': {'runtime': 13}}
Run cool-pond-357 has no macro f1
{'_wandb': {'runtime': 15}}
Run hearty-bee-356 has no macro f1
{'_wandb': {'runtime': 12}}
Run light-silence-355 has no macro f1
{'_wandb': {'runtime': 33}}
Run silver-feather-354 has no macro f1
{'_wandb': {'runtime': 60}}


In [26]:
import pandas as pd

df = pd.DataFrame(data)

df["model"] = df["model"].map(lambda x: renames[x])

# Group by model and task

grouped = df.groupby(["task", "model"])

# Get mean macro f1 plus standard deviation in a single line with +- inbetween

df_results = (grouped.mean() * 100).round(1).astype(str) + " +- " + (grouped.std() * 100).round(1).astype(str)

df_results = df_results.unstack("task")
df_results.columns = df_results.columns.droplevel(0)

df_results

task,emotion,hate_speech,irony,sentiment
model,,,,
AlBERTo,72.0 +- 1.3,88.1 +- 0.4,53.7 +- 0.6,57.8 +- 0.7
BERT-it,73.6 +- 4.0,92.4 +- 0.4,62.0 +- 4.4,61.4 +- 0.9
Electra-it,64.7 +- 7.7,87.8 +- 3.0,50.0 +- 6.5,62.3 +- 0.7
RoBERTuito,64.1 +- 3.0,92.6 +- 0.3,55.6 +- 3.6,55.2 +- 2.8
UmBERTo,69.7 +- 4.6,87.3 +- 0.4,60.0 +- 2.2,62.6 +- 1.1


In [27]:
print(df_results.to_markdown())

| model      | emotion     | hate_speech   | irony       | sentiment   |
|:-----------|:------------|:--------------|:------------|:------------|
| AlBERTo    | 72.0 +- 1.3 | 88.1 +- 0.4   | 53.7 +- 0.6 | 57.8 +- 0.7 |
| BERT-it    | 73.6 +- 4.0 | 92.4 +- 0.4   | 62.0 +- 4.4 | 61.4 +- 0.9 |
| Electra-it | 64.7 +- 7.7 | 87.8 +- 3.0   | 50.0 +- 6.5 | 62.3 +- 0.7 |
| RoBERTuito | 64.1 +- 3.0 | 92.6 +- 0.3   | 55.6 +- 3.6 | 55.2 +- 2.8 |
| UmBERTo    | 69.7 +- 4.6 | 87.3 +- 0.4   | 60.0 +- 2.2 | 62.6 +- 1.1 |


In [28]:
mean_df = pd.DataFrame(data).groupby(["model", "task"]).mean()
std_df = pd.DataFrame(data).groupby(["model", "task"]).std()

# Concat mean and std
df = pd.concat([mean_df, std_df], axis=1)

df.columns = ["mean macro f1", "std macro f1"]

df = df * 100

df

mean macro f1  \
model                                              task                         
Musixmatch/umberto-commoncrawl-cased-v1            emotion          69.724572   
                                                   hate_speech      87.262057   
                                                   irony            60.035865   
                                                   sentiment        62.601565   
dbmdz/bert-base-italian-xxl-uncased                emotion          73.588963   
                                                   hate_speech      92.383168   
                                                   irony            61.974809   
                                                   sentiment        61.391095   
dbmdz/electra-base-italian-xxl-cased-discriminator emotion          64.682508   
                                                   hate_speech      87.751349   
                                                   irony            49.958163   
                                                   sentiment        62.265987   
m-polignano-uniba/bert_uncased_L-12_H-768_A-12_... emotion          72.020791   
                                                   hate_speech      88.051860   
                                                   irony            53.664187   
                                                   sentiment        57.771450   
pysentimiento/robertuito-base-uncased              emotion          64.054664   
                                                   hate_speech      92.632221   
                                                   irony            55.600531   
                                                   sentiment        55.161193   

                                                                std macro f1  
model                                              task                       
Musixmatch/umberto-commoncrawl-cased-v1            emotion          4.600150  
                                                   hate_speech      0.363911  
                                                   irony            2.183473  
                                                   sentiment        1.063719  
dbmdz/bert-base-italian-xxl-uncased                emotion          4.019144  
                                                   hate_speech      0.409217  
                                                   irony            4.378350  
                                                   sentiment        0.863827  
dbmdz/electra-base-italian-xxl-cased-discriminator emotion          7.675118  
                                                   hate_speech      2.963175  
                                                   irony            6.492525  
                                                   sentiment        0.704775  
m-polignano-uniba/bert_uncased_L-12_H-768_A-12_... emotion          1.339000  
                                                   hate_speech      0.406965  
                                                   irony            0.626334  
                                                   sentiment        0.665071  
pysentimiento/robertuito-base-uncased              emotion          2.957688  
                                                   hate_speech      0.271326  
                                                   irony            3.560796  
                                                   sentiment        2.842376

In [29]:
df.reset_index().to_csv("../data/results_it.csv", index=False)

In [30]:
df = pd.read_csv("../data/results_it.csv").set_index(["model", "task"])

In [31]:
df.loc[("pysentimiento/robertuito-base-uncased", "emotion"), "mean macro f1"]

64.05466397603354